# Realtime-emotion-detectionusing-videotest
I hope that by analysing facial expressions, we can better understand and respond to people's emotional states. The code provides a simple and practical base framework to help developers quickly build and deploy applications for facial emotion analysis. By combining computer vision and deep learning techniques, we can create a more intelligent and human interaction experience.

This code is a deep learning based facial emotion analysis application. It uses computer vision and machine learning techniques to detect faces captured by the camera and to predict the emotion category of facial expressions. The code performs face detection using OpenCV by loading pre-trained models and weights, and pre-processes and classifies the detected face regions for emotion.

## Reference
[CSDN](https://blog.csdn.net/weixin_42143481/article/details/105771183?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168681998016800225516119%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168681998016800225516119&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-105771183-null-null.142^v88^insert_down38v5,239^v2^insert_chatgpt&utm_term=colab%E6%91%84%E5%83%8F%E5%A4%B4&spm=1018.2226.3001.4187)

https://morioh.com/p/801c509dda99

https://github.com/Dhanush45/Realtime-emotion-detectionusing-python

# Importing necessary libraries for plotting and image processing

In [ ]:
import os                       #For interaction with the operating system
import cv2                       #OpenCV library for image processing
import numpy as np                  #For numerical calculations
from keras.models import model_from_json       #Functions for loading models from JSON files in Keras
from keras.preprocessing import image        #Image pre-processing tools are provided

from IPython.display import display, Javascript   #For displaying images or other content，Allow JavaScript code to be executed
from google.colab.output import eval_js       #For evaluating JavaScript code
from base64 import b64decode             #For decoding Base64 encoded data

from IPython.display import HTML, Audio       #Module for displaying HTML content and playing audio

import matplotlib.pyplot as plt           #For creating charts
from google.colab.patches import cv2_imshow     #Function from the google.colab.patches module for displaying images in Google Colab.
from google.colab.output import eval_js
from base64 import b64decode             #For decoding Base64 encoded data
import numpy as np
import io                       #For input/output operations
from PIL import Image




# For capturing video frames in web pages and converting them to images

In [ ]:
VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] =[video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""
#A string variable containing HTML and JavaScript is defined to display the video in the web page.
#Get the video element in a web page.
#Use the navigator.mediaDevices.getUserMedia method to request user authorization to access the video device and assign the video stream to the srcObject property of the video element to play the video in the web page
#Create a Promise object to be triggered when the user clicks on the video. In the click event handler, create a canvas element to draw the video frame. Get the width and height of the video element and set the size of the canvas. Draw the video frame to the canvas using the drawImage method of the 2D context. Stop the video stream and replace the video element with the canvas. Finally, convert the canvas to Base64 encoding of the image and use the result as a parsed value for the Promise.
#The purpose of the entire code block is to capture video frames and convert them into image data for subsequent processing or display.

# Capture a photo from a video and return its image data

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8, size=(800,600)):  #A function called take_photo is defined which accepts as arguments the filename, image quality and size, with default values of 'photo.jpg', 0.8 and (800,600) respectively.
  display(HTML(VIDEO_HTML % (size[0],size[1],quality)))       #Display the HTML content in a Jupyter/Colab notebook, using the previously defined string VIDEO_HTML and inserting the values of size and quality into the HTML.
  data = eval_js("data")                       #Get the value of the data variable defined earlier by executing the JavaScript code eval_js("data").
  binary = b64decode(data.split(',')[1])               #Separate the values of the data variable by commas and decode the Base64 encoded image data.
  f = io.BytesIO(binary)                       #Store the decoded image data in the io.BytesIO object.
  return np.asarray(Image.open(f))                  #Open the image in f and return it after converting it to a NumPy array.
                                     #The function does this by displaying the video stream and waiting for the user to click to capture a photo, then converting the photo to a NumPy array to be returned for further processing or analysis.


In [ ]:
#load model
model = model_from_json(open("fer.json", "r").read())
#load weights
model.load_weights('fer.h5')

In [ ]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#A cascade classifier file called 'haarcascade_frontalface_default.xml' is loaded. A cascade classifier is an algorithm used for object detection and this file is specifically used for detecting faces. face_haar_cascade variable will hold this cascade classifier object and can be used to detect faces in images or videos.

In [ ]:
cap=cv2.VideoCapture(0)                              #Open the default camera device and create a VideoCapture object and assign it to the cap variable.

while True:                                    #Enter an infinite loop for continuous processing of video frames
    test_img=take_photo()                           # captures frame and returns boolean value and captured image

    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)           #Converts the captured colour image into a greyscale image for subsequent face detection.



    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5) #Use the face_haar_cascade cascade classifier object to detect faces in a grey-scale image, returning the coordinates of the rectangular region of the face.


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]                   #cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))

        img_pixels = roi_gray.astype('float32')
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255.0
                                        #For each face detected, a blue rectangular box is drawn and the region of interest (ROI), the face region, is truncated. The ROI is then resized to 48x48 and the data is normalised.

        predictions = model.predict(img_pixels)             #Emotion prediction on normalised face images using a pre-trained model mod.

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]
                                        #Find the sentiment category with the highest probability in the predicted outcome and convert it to the corresponding sentiment label.
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2) #Plotting the predicted sentiment labels on the image.

    resized_img = cv2.resize(test_img, (1000, 700))
                                        #cv2.imshow('Facial emotion analysis ',resized_img)
    plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
                                        # as opencv loads in BGR format by default, we want to show it in RGB.
    plt.show()

    if cv2.waitKey(10) == ord('q'):                   #wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows
#Free up camera resources and close all windows.